In [ ]:
# system tools
import os
import sys
from io import StringIO
import shutil

# web scrapping tools
import bs4 as bs
import urllib
import urllib.request
import pandas as pd

#---

def get_data_url(event: str) -> str:
    '''Takes an event name and returns the URL for the data page.'''

    event = event.split('-') # split the event name into its components, seperated by '-'
    year = event[0]  # the first component is the year
    region = event[1].lower()  # the second component is region (e.g., blg or gd), which we need to make lower case.
    number = event[2]  #
    url = f'https://www.astrouw.edu.pl/ogle/ogle4/ews/{year}/{region}-{number}/phot.dat'

    return url

def get_param_url(event: str) -> str:
    '''Takes an event name and returns the URL for the parameter page.'''

    event = event.split('-') # split the event name into its components, seperated by '-'
    year = event[0]  # the first component is the year
    region = event[1].lower()  # the second component is region (e.g., blg or gd), which we need to make lower case.
    number = event[2]  #
    url = f'https://www.astrouw.edu.pl/ogle/ogle4/ews/{year}/{region}-{number}/params.dat'

    return url

def fetch_event_data(url: str) -> pd.DataFrame:
    '''Takes a url and returns the data as a pandas dataframe.'''

    # Read the data from the URL
    response = urllib.request.urlopen(url)
    data = response.read().decode('utf-8')

    # Convert the data to a pandas DataFrame
    #df = pd.read_csv(StringIO(data), delim_whitespace=True, header=None, names=['HJD', 'I magnitude', 'magnitude error', 'seeing', 'sky level'])
    df = pd.read_csv(StringIO(data), sep=r'\s+', header=None, names=['HJD', 'I magnitude', 'magnitude error', 'seeing', 'sky level'])

    return df

#---

def fetch_table_data(url):
    '''Takes a URL and returns the first table as a pandas DataFrame.'''
    source = urllib.request.urlopen(url).read().decode('utf-8')
    df = pd.read_csv(StringIO(source), sep=r'\s+', header=5, names=['Parameter', 'Value', 'Error'])

    return df






In [ ]:


event = '2012-BLG-0406'
url_data = get_data_url(event)
url_param = get_param_url(event)
data = fetch_event_data(url_data)
data['HJD'] = data['HJD'] - 2450000
params = fetch_table_data(url_param)
data_columns = ['HJD', 'I magnitude', 'magnitude error']
params_columns = ['Parameter', 'Value', 'Error']

data.to_csv(
    '../gulls_stuff/%s_data.txt'%event,          # output path
    columns=data_columns,  # only these columns
    sep='\t',              # delimiter (e.g. '\t', ',' or ' ')
    index=False,           # don’t write row numbers
    header=['HJD-2450000', 'I_Mag', 'Err']           # include the column names
)
params.to_csv(
    '../gulls_stuff/%s_params.txt'%event,          # output path
    columns=params_columns,  # only these columns
    sep='\t',              # delimiter (e.g. '\t', ',' or ' ')
    index=False,           # don’t write row numbers
    header=True            # include the column names
)

In [ ]:
event = '2012-BLG-0406'
time, mag, magerr = np.loadtxt('../gulls_stuff/%s_data.txt'%event, delimiter='\t', skiprows=1, unpack=True)

plt.figure(figsize=(8, 5))
plt.errorbar(time,mag,yerr=magerr,fmt='x',color='black')
plt.xlabel('HJD')
plt.ylabel('I magnitude')
plt.gca().invert_yaxis()
plt.title('OGLE-%s' % event)

                  